In [1]:
#from https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21

Text Cleaning

In [2]:
#Clean Text, return list of tokens
import sys
!{sys.executable} -m pip install spacy
!{sys.executable}pip install -U pip setuptools wheel
!{sys.executable}pip install -U spacy
!{sys.executable} -m spacy download en_core_web_sm
!{sys.executable} -m spacy.en.download
!{sys.executable}pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz
import spacy
import en_core_web_sm
#nlp = en_core_web_sm.load()
nlp = spacy.load("en_core_web_sm")
from spacy.lang.en import English
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

#Find meanings of words, synonyms, antonyms, etc.
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

#Get root words (lemmatizer)
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

#filter out stop words
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

#topic modeling
def prepare_text_for_lda(text):
    words_to_remove = ['student', 'call', 'contact', 'regard', 'advise']
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    tokens = [token for token in tokens if token not in words_to_remove]
    return tokens
;

'C:\Users\eamador\Anaconda3\python.exepip' is not recognized as an internal or external command,
operable program or batch file.
'C:\Users\eamador\Anaconda3\python.exepip' is not recognized as an internal or external command,
operable program or batch file.


[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


C:\Users\eamador\Anaconda3\python.exe: Error while finding module specification for 'spacy.en.download' (ModuleNotFoundError: No module named 'spacy.en')
'C:\Users\eamador\Anaconda3\python.exepip3' is not recognized as an internal or external command,
operable program or batch file.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\eamador\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eamador\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


''

In [3]:
#TODO get data from oracle instead of csv

In [4]:
#print list of converted text elements
import random
text_data = []
with open('Case_Descriptions.csv') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            text_data.append(tokens)
;

''

LDA

In [5]:
#Create dictionary, convert to bag-of-words
!{sys.executable} -m pip install --upgrade gensim
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim');

Requirement already up-to-date: gensim in c:\users\eamador\anaconda3\lib\site-packages (3.8.3)


In [6]:
#Find Topics
import gensim
NUM_TOPICS = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.055*"parent" + 0.049*"within" + 0.044*"handle" + 0.038*"blank"')
(1, '0.038*"assistance" + 0.034*"email" + 0.028*"instructor" + 0.026*"refer"')
(2, '0.091*"resolve" + 0.077*"parent" + 0.038*"course" + 0.026*"feedback"')
(3, '0.052*"button" + 0.048*"schedule" + 0.041*"reschedule" + 0.037*"assessment"')
(4, '0.085*"score" + 0.075*"need" + 0.049*"action" + 0.038*"post"')
(5, '0.076*"assessment" + 0.028*"reschedule" + 0.027*"schedule" + 0.027*"launch"')
(6, '0.084*"event" + 0.042*"generate" + 0.042*"queue" + 0.027*"proctor"')
(7, '0.054*"schedule" + 0.049*"mentor" + 0.045*"request" + 0.036*"assessment"')
(8, '0.066*"proctor" + 0.037*"reschedule" + 0.036*"examity" + 0.036*"connect"')
(9, '0.026*"issue" + 0.019*"submit" + 0.017*"requirement" + 0.016*"state"')
(10, '0.061*"email" + 0.046*"reschedule" + 0.036*"launch" + 0.029*"unable"')
(11, '0.101*"assessment" + 0.064*"schedule" + 0.037*"issue" + 0.034*"confirm"')
(12, '0.090*"attempt" + 0.049*"approval" + 0.045*"inform" + 0.040*"cours

Visualize Saliency and Relevance

In [16]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')
!{sys.executable} -m pip install pyLDAvis
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)